## Optimización de Hiperparámetros

El siguiente ejemplo implementa la solución a problema básico de clasificación usando una red neuronal (MLPClassifier), entrenada con el conjunto de datos digits (dígitos manuscritos), y con optimización bayesiana de hiperparámetros usando Optuna.

### Importar librerías

In [ ]:
try:
    # Librería de optimización bayesiana para ajuste de hiperparámetros
    import optuna
    print("Librería 'optuna' instalada y cargada correctamente")
    print(optuna.__version__)
except:
    print("Librería 'optuna' no instalada, se procederá con la instalación")
    !pip install scikit-learn optuna
    # Librería de optimización bayesiana para ajuste de hiperparámetros
    import optuna
    print(optuna.__version__)

# Conjunto de datos de dígitos manuscritos incluido en scikit-learn
from sklearn.datasets import load_digits

# Función para dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split, cross_val_score

# Clasificador de perceptrón multicapa (red neuronal) de scikit-learn
from sklearn.neural_network import MLPClassifier

# Normalizador de características (media = 0, varianza = 1)
from sklearn.preprocessing import StandardScaler

# Permite crear un pipeline (flujo de procesamiento) de pasos encadenados
from sklearn.pipeline import make_pipeline

# Métrica para evaluar la precisión del modelo (proporción de aciertos)
from sklearn.metrics import accuracy_score

# Ignorar las advertencias
import warnings
warnings.filterwarnings("ignore")

### Cargar datos y dividirlos en entrenamiento y pruebas

In [ ]:
# Carga el conjunto de datos de dígitos manuscritos (8x8 píxeles) de scikit-learn
digits = load_digits()

# Separa las características (X) y las etiquetas (y) del conjunto de datos
X, y = digits.data, digits.target

# Divide el conjunto de datos en entrenamiento (80%) y prueba (20%)
# El parámetro random_state asegura que la división sea reproducible
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Función objetivo para Optuna

In [ ]:
def objective(trial):
    # Definición del espacio de búsqueda
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(64,), (100,), (64, 64), (100, 50)])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    solver = trial.suggest_categorical('solver', ['adam', 'sgd'])
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-1)
    learning_rate_init = trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-1)

    # Crear pipeline con estandarización
    clf = make_pipeline(
        StandardScaler(),
        MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,
            activation=activation,
            solver=solver,
            alpha=alpha,
            learning_rate_init=learning_rate_init,
            max_iter=100,
            random_state=42
        )
    )

    # Validación cruzada
    score = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')
    return score.mean()

### Ejecutar la optimización

In [ ]:
%%time

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(study.best_params)

### Evaluar el mejor modelo en el conjunto de prueba

In [ ]:
best_params = study.best_params
best_model = make_pipeline(
    StandardScaler(),
    MLPClassifier(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        solver=best_params['solver'],
        alpha=best_params['alpha'],
        learning_rate_init=best_params['learning_rate_init'],
        max_iter=100,
        random_state=42
    )
)

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Precisión en el conjunto de prueba: {acc:.4f}")

### ¿Qué hace este ejemplo?

* Usa el conjunto digits (1797 imágenes de dígitos manuscritos).
* Crea un MLPClassifier con diferentes combinaciones de hiperparámetros (capas ocultas, activación, alpha, tasa de aprendizaje...).
* Usa Optuna para optimizar esos hiperparámetros mediante validación cruzada.
* Evalúa el modelo final con los mejores parámetros sobre el conjunto de prueba.